@Copyright IQIYI 2021
http://challenge.ai.iqiyi.com/

In [19]:
import pandas as pd
import numpy as np
from itertools import groupby
from tqdm import tqdm

In [2]:
input_dir = "../data/"
output_dir = "../process_data/"

In [3]:
launch = pd.read_csv(input_dir + "app_launch_logs.csv")
launch.date.min(), launch.date.max()

(100, 222)

In [4]:
launch.head()

,user_id,launch_type,date
0,10157996,0,129
1,10139583,0,129
2,10277501,0,129
3,10099847,0,129
4,10532773,0,129


In [5]:
launch_grp = launch.groupby("user_id").agg(
    launch_date=("date", list),
    launch_type=("launch_type", list)
).reset_index()
launch_grp

,user_id,launch_date,launch_type
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,10000001,"[118, 117, 114]","[0, 0, 0]"
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]"
3,10000003,"[144, 144]","[1, 0]"
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]"
...,...,...,...
599995,10599995,"[196, 174, 197, 190, 189, 155, 201, 175, 165, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]"
599996,10599996,"[185, 209, 207, 187, 189, 213, 184, 199, 210, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
599997,10599997,"[115, 159, 160, 166, 161, 157, 163, 158, 134, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
599998,10599998,"[137, 160, 182, 159, 107]","[0, 0, 0, 0, 0]"


In [6]:
# generating a sample for each user, a sample should has an anchor date for label
def choose_end_date(launch_date):
    n1, n2 = min(launch_date), max(launch_date)
    if n1 < n2 - 7:
        end_date = np.random.randint(n1, n2 - 7)
    else:
        end_date = np.random.randint(100, 222 - 7)
    return end_date
launch_grp["end_date"] = launch_grp.launch_date.apply(choose_end_date)
launch_grp

,user_id,launch_date,launch_type,end_date
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",178
1,10000001,"[118, 117, 114]","[0, 0, 0]",211
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]",153
3,10000003,"[144, 144]","[1, 0]",206
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]",176
...,...,...,...,...
599995,10599995,"[196, 174, 197, 190, 189, 155, 201, 175, 165, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]",178
599996,10599996,"[185, 209, 207, 187, 189, 213, 184, 199, 210, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",204
599997,10599997,"[115, 159, 160, 166, 161, 157, 163, 158, 134, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",164
599998,10599998,"[137, 160, 182, 159, 107]","[0, 0, 0, 0, 0]",148


In [7]:
def get_label(row):
    launch_list = row.launch_date
    end = row.end_date
    label = sum([1 for x in set(launch_list) if end < x < end+8])
    return label
launch_grp["label"] = launch_grp.apply(get_label, axis=1)
launch_grp

,user_id,launch_date,launch_type,end_date,label
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",178,2
1,10000001,"[118, 117, 114]","[0, 0, 0]",211,0
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]",153,1
3,10000003,"[144, 144]","[1, 0]",206,0
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]",176,0
...,...,...,...,...,...
599995,10599995,"[196, 174, 197, 190, 189, 155, 201, 175, 165, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]",178,0
599996,10599996,"[185, 209, 207, 187, 189, 213, 184, 199, 210, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",204,4
599997,10599997,"[115, 159, 160, 166, 161, 157, 163, 158, 134, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",164,2
599998,10599998,"[137, 160, 182, 159, 107]","[0, 0, 0, 0, 0]",148,0


In [9]:
launch_grp.label.value_counts()

0    302979
1    126235
2     57926
3     33384
4     23545
7     20242
5     19022
6     16667
Name: label, dtype: int64

In [10]:
train = launch_grp[["user_id", "end_date", "label"]]
train

,user_id,end_date,label
0,10000000,178,2
1,10000001,211,0
2,10000002,153,1
3,10000003,206,0
4,10000004,176,0
...,...,...,...
599995,10599995,178,0
599996,10599996,204,4
599997,10599997,164,2
599998,10599998,148,0


In [11]:
# reading test data
test = pd.read_csv(input_dir + "test-a.csv")
test["label"] = -1
test

,user_id,end_date,label
0,10007813,205,-1
1,10052988,210,-1
2,10279068,200,-1
3,10546696,216,-1
4,10406659,183,-1
...,...,...,...
14996,10355586,205,-1
14997,10589773,210,-1
14998,10181954,218,-1
14999,10544736,164,-1


In [12]:
# concat train and test data
data = pd.concat([train, test], ignore_index=True)
data

,user_id,end_date,label
0,10000000,178,2
1,10000001,211,0
2,10000002,153,1
3,10000003,206,0
4,10000004,176,0
...,...,...,...
614996,10355586,205,-1
614997,10589773,210,-1
614998,10181954,218,-1
614999,10544736,164,-1


# launch data process

In [13]:
# append test data to launch_grp
launch_grp = launch_grp.append(
    test.merge(launch_grp[["user_id", "launch_type", "launch_date"]], how="left", on="user_id")
)
launch_grp

,user_id,launch_date,launch_type,end_date,label
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",178,2
1,10000001,"[118, 117, 114]","[0, 0, 0]",211,0
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]",153,1
3,10000003,"[144, 144]","[1, 0]",206,0
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]",176,0
...,...,...,...,...,...
14996,10355586,"[118, 115, 124]","[0, 1, 0]",205,-1
14997,10589773,"[183, 144, 173, 167, 203, 204, 178, 155, 177, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",210,-1
14998,10181954,"[152, 155, 153, 144, 150, 143, 158, 151, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",218,-1
14999,10544736,"[138, 117, 105, 148, 113, 116, 115, 158, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",164,-1


In [16]:
# get latest 32 days([end_date-31, end_date]) launch type sequence
# 0 for not launch, 1 for launch_type=0, and 2 for launch_type=1
def gen_launch_seq(row):
    seq_sort = sorted(zip(row.launch_type, row.launch_date), key=lambda x: x[1])
    seq_map = {k: max(g)[0] + 1 for k, g in groupby(seq_sort, lambda x: x[1])}
    end = row.end_date
    seq = [seq_map.get(x, 0) for x in range(end-31, end+1)]
    return seq
launch_grp["launch_seq"] = launch_grp.apply(gen_launch_seq, axis=1)
launch_grp

,user_id,launch_date,launch_type,end_date,label,launch_seq
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",178,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,10000001,"[118, 117, 114]","[0, 0, 0]",211,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]",153,1,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,10000003,"[144, 144]","[1, 0]",206,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]",176,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."
...,...,...,...,...,...,...
14996,10355586,"[118, 115, 124]","[0, 1, 0]",205,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14997,10589773,"[183, 144, 173, 167, 203, 204, 178, 155, 177, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",210,-1,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ..."
14998,10181954,"[152, 155, 153, 144, 150, 143, 158, 151, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",218,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14999,10544736,"[138, 117, 105, 148, 113, 116, 115, 158, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",164,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [25]:
#计算end_date前x天的用户登陆天数
x_list = [3,7,11,15,19,23,27,31]
for x in tqdm(x_list):
    launch_grp[f'{x}_before_end_date'] = launch_grp['launch_seq'].apply(lambda seq: np.sum(seq[-x:]))

100%|███████████████████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.19s/it]


In [27]:
x_feature_list = [col for col in launch_grp.columns if 'before' in col]
data = data.merge(
    launch_grp[["user_id", "end_date", "label"]+x_feature_list],
    on=["user_id", "end_date", "label"],
    how="left"
)
data

,user_id,end_date,label,launch_seq,3_before_end_date,7_before_end_date,11_before_end_date,15_before_end_date,19_before_end_date,23_before_end_date,27_before_end_date,31_before_end_date
0,10000000,178,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,1,1,1,1,1
1,10000001,211,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0
2,10000002,153,1,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,2,2,2,4,4
3,10000003,206,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0
4,10000004,176,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...",0,0,0,0,0,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
614996,10355586,205,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0
614997,10589773,210,-1,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,7,11,14,17,21,24,28
614998,10181954,218,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0
614999,10544736,164,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,3,3,4,4,5,5


In [66]:
# finally
data.loc[data.label >= 0].to_csv(output_dir + "train_data_lgb.txt", sep="\t", index=False)
data.loc[data.label < 0].to_csv(output_dir + "test_data_lgb.txt", sep="\t", index=False)

# LGB

In [75]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [67]:
train_df = data.loc[data.label >= 0].reset_index(drop=True)
test_df = data.loc[data.label < 0].reset_index(drop=True)

In [121]:
drop_feature_list = ['end_date','launch_seq','label','playtime_seq','duration_prefer','interact_prefer']
label = 'label'
feature = [x for x in train_df.columns if x not in drop_feature_list]
feature = [x for x in train_df.columns if 'before' in x]
print(len(feature))

8


In [122]:
#lgb
def lgb_custom_metric(y_true,y_pre):
    y_pre = y_pre.get_label()
    score = 100*(1-np.mean((np.abs(y_pre.reshape(-1) - y_true.reshape(-1))/7)))
    return 'self-metric', score, True
#自己用的
def custom_metric(y_true,y_pre):
    score = 100*(1-np.mean((np.abs(y_pre.reshape(-1) - y_true.reshape(-1))/7)))
    return score

In [123]:
n_fold = 5
#lgb参数
lgb_params = {
    "boosting_type": "gbdt",
    "objective": "mae",
    "learning_rate": 0.01,
    "max_depth": -1,
    "num_leaves": 31,
    "feature_fraction": 1,
    "bagging_fraction": 1,     
    'min_data_in_leaf': 100,  
    'bagging_freq': 6, 
    "nthread":-1
    
}
y_val = np.zeros((train_df.shape[0]))
y_test = np.zeros((test_df.shape[0]))
score_list = []
skf = StratifiedKFold(n_splits=n_fold,shuffle=True,random_state=42)
for train_index,valid_index in skf.split(train_df[feature],train_df[label]):
    x_train,x_valid,y_train,y_valid = train_df[feature].iloc[train_index],train_df[feature].iloc[valid_index],train_df[label].iloc[train_index],train_df[label].iloc[valid_index]

    train_data = lgb.Dataset(x_train,label=y_train)
    valid_data = lgb.Dataset(x_valid,label=y_valid)
    
    model = lgb.train(lgb_params,train_data,valid_sets=[valid_data],num_boost_round=10000,verbose_eval=50, early_stopping_rounds = 50,feval=lgb_custom_metric)
    
    y_val[valid_index] = model.predict(x_valid)
    y_test += np.array(model.predict(test_df[feature])/n_fold)

cv_score = custom_metric(y_val,train_df[label].values)
print(f'local cv: {cv_score}')

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007054 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 149
[LightGBM] [Info] Number of data points in the train set: 480000, number of used features: 8
Training until validation scores don't improve for 50 rounds
[50]	valid_0's l1: 1.05692	valid_0's self-metric: 84.9011
[100]	valid_0's l1: 0.953004	valid_0's self-metric: 86.3857
[150]	valid_0's l1: 0.907622	valid_0's self-metric: 87.034
[200]	valid_0's l1: 0.888032	valid_0's self-metric: 87.3138
[250]	valid_0's l1: 0.879073	valid_0's self-metric: 87.4418
[300]	valid_0's l1: 0.875249	valid_0's self-metric: 87.4964
[350]	valid_0's l1: 0.873422	valid_0's self-metric: 87.5225
[400]	valid_0's l1: 0.872512	valid_0's self-metric: 87.5355
[450]	valid_0's l1: 0.871902	valid_0's self-metric: 87.5443
[500]	valid_0's l1: 0.871573	valid_0's self-met

[450]	valid_0's l1: 0.870572	valid_0's self-metric: 87.5633
[500]	valid_0's l1: 0.869946	valid_0's self-metric: 87.5722
[550]	valid_0's l1: 0.869664	valid_0's self-metric: 87.5762
[600]	valid_0's l1: 0.869549	valid_0's self-metric: 87.5779
[650]	valid_0's l1: 0.869479	valid_0's self-metric: 87.5789
[700]	valid_0's l1: 0.869455	valid_0's self-metric: 87.5792
[750]	valid_0's l1: 0.869442	valid_0's self-metric: 87.5794
[800]	valid_0's l1: 0.869435	valid_0's self-metric: 87.5795
[850]	valid_0's l1: 0.86943	valid_0's self-metric: 87.5796
[900]	valid_0's l1: 0.869428	valid_0's self-metric: 87.5796
[950]	valid_0's l1: 0.869426	valid_0's self-metric: 87.5796
[1000]	valid_0's l1: 0.869425	valid_0's self-metric: 87.5796
[1050]	valid_0's l1: 0.869424	valid_0's self-metric: 87.5797
[1100]	valid_0's l1: 0.869424	valid_0's self-metric: 87.5797
[1150]	valid_0's l1: 0.869424	valid_0's self-metric: 87.5797
[1200]	valid_0's l1: 0.869424	valid_0's self-metric: 87.5797
[1250]	valid_0's l1: 0.869424	valid_

[2100]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2150]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2200]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2250]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2300]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2350]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2400]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2450]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2500]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2550]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2600]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2650]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2700]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2750]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2800]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2850]	valid_0's l1: 0.872438	valid_0's self-metric: 87.5366
[2900]	valid_0's l1: 0.8

In [124]:
res = pd.DataFrame()
res['user_id'] = test_df['user_id']
res['pre'] = y_test
res.to_csv(f"lgb_base_{cv_score}.csv", index=False, header=False, float_format="%.2f")

In [125]:
feature_imp_df = pd.DataFrame()
feature_imp_df['fea_name'] = model.feature_name()
feature_imp_df['fea_imp'] = model.feature_importance()
feature_imp_df = feature_imp_df.sort_values('fea_imp',ascending=False)

In [126]:
feature_imp_df

,fea_name,fea_imp
7,31_before_end_date,35524
5,23_before_end_date,17385
1,7_before_end_date,15899
0,3_before_end_date,14383
3,15_before_end_date,10904
6,27_before_end_date,7194
4,19_before_end_date,4010
2,11_before_end_date,3031
